[GDG Ahlen / Incremental design of LLM-powered agentic applications](https://www.youtube.com/watch?v=uIQlMSX5gx4)

Resources:
- [Google AI Studio](https://python.langchain.com/docs/integrations/chat/google_generative_ai/)
    - need to get `GOOGLE_API_KEY`
- [Finage API](https://finage.co.uk/docs/api/us-stocks#stock-market-previous-close)
    - need to get `FINAGE_API_KEY`
- [LangGraph: ReAct example](https://langchain-ai.github.io/langgraph/#example)


# Module 3: ReAct pattern using LangGraph

## Setup Gemini in LangChain and import LangGraph components

Important to set Experimental model of Gemini in `ChatGoogleGenerativeAI.model` 

In [53]:
# Import env variables from `.env` file
from dotenv import load_dotenv
load_dotenv()

# Check for Google GEMINI API KEY
import os
assert "GOOGLE_API_KEY" in os.environ

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.tools import tool

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition

llm = ChatGoogleGenerativeAI(
    max_tokens=None,
    model="gemini-2.0-flash-exp",
    temperature=1,
    top_k=1,
    top_p=0.9
)

# Set system prompt
SYSTEM_PROMPT = SystemMessage(content="""
    You are investment analyst equipped with a tool which gets previous day's closing price for a stock symbol.
""")



## Setup LangChain tools

In [54]:
import requests
from typing import Dict

@tool
def get_stock_data(symbol: str) -> Dict:
    """
    Get previous day's closing price for a stock symbol using Finage API
    
    Args:
        symbol (str): Stock symbol (e.g. 'AAPL' for Apple)
        
    Returns:
        Dict: Response data containing previous close price and other details
    """
    api_key = os.getenv("FINAGE_API_KEY")
    if not api_key:
        raise ValueError("FINAGE_API_KEY not found in environment variables")
        
    url = f"https://api.finage.co.uk/agg/stock/prev-close/{symbol}"
    
    params = {
        "apikey": api_key
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error fetching stock data: {str(e)}")


tools = [get_stock_data]

llm_with_tools = llm.bind_tools(tools)

## Configure LangGraph methods

In [55]:
from typing import Literal

# Define the function that determines whether to continue or not
# def should_continue(state: MessagesState) -> Literal["tools", END]:
#     messages = state['messages']
#     last_message = messages[-1]

#     # If the LLM makes a tool call, then we route to the "tools" node
#     if last_message.tool_calls:
#         return "tools"
    
#     # Otherwise, we stop (reply to the user)
#     return END
# or use method `tools_condition` imported from langgraph.prebuilt


# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = llm_with_tools.invoke([SYSTEM_PROMPT] + messages)

    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

## Configure LangGraph nodes

![ReAct pattern](./images/react-pattern.jpeg)

In [56]:
# Define a new graph
workflow = StateGraph(MessagesState)

AGENT_NODE = "agent"
TOOLS_NODE = "tools"

# Define the two nodes we will cycle between
tools_node = ToolNode(tools)
workflow.add_node(AGENT_NODE, call_model)
workflow.add_node(TOOLS_NODE, tools_node)

# Set the entrypoint as `agent`
workflow.add_edge(START, AGENT_NODE)

# Add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    AGENT_NODE,
    # Next, we pass in the function that will determine which node is called next.
    tools_condition,
)

# We now add a normal edge from `tools` to `agent`.
workflow.add_edge(TOOLS_NODE, AGENT_NODE)

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it into LangChain Runnable
# (optionally) passing the memory when compiling the graph
agent = workflow.compile(checkpointer=checkpointer)

## Invoke LangGraph agent

In [57]:
# Assign unique id to new conversation
THREAD_ID = 42

# Use the agent
config = {"configurable": {"thread_id": THREAD_ID}}
final_state = agent.invoke(
    {"messages": [HumanMessage(content="What is the price for Apple?")]},
    config
)

# Display final respone
final_state["messages"][-1].content

"The previous day's closing price for Apple (AAPL) was 235.33."